In [54]:
import asyncio
from playwright.async_api import async_playwright
import time

In [2]:
_dataset = "Australian Census Longitudinal Data"
_table = "Australian Census Longitudinal Dataset, 2016-2021"

In [150]:
import pandas as pd

df = pd.read_csv("../Data/LGAs.csv")

lgas = df["LGA (EN)"].tolist()

In [ ]:
constant_vars = {
    "var1": {
        "subdirs" : ["2016", "2016 Person", "2016 Voluntary and Unpaid work"],
        "name" : "Unpaid Child Care in 2016",
        "level" : "Unpaid Child Care in 2016",
        "add to" : "Row"
   }
}

In [51]:
button_titles = {
    "Row" : "Add selected items to the table row",
    "Column" : "Add selected items to the table column",
    "Wafer": "Add selected items to the table wafer",
    "Filter": "Add selected items to the table filter"
}

In [ ]:
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless=False)
page = await browser.new_page()
await page.goto('https://tablebuilder.abs.gov.au/webapi/jsf/login.xhtml')

await page.get_by_label("User ID is a number:").fill("1234567")
await page.get_by_label("Password:").fill("Yfddfdfdfdfd")
await page.get_by_role("button", name="Log in").click()

#await page.locator("//*[contains(@class, 'treeNodeElement')]", has = page.get_by_title(_dataset)).locator("//*[contains(@class, 'treeNodeExpander collapsed')]").click()
#await page.get_by_title(_table).dblclick()

#await page.wait_for_load_state()


#|//*[contains(@class, 'treeNodeExpander expanded')]

In [133]:
iterable_var = {
    "subdirs" : ["Geographical Areas (Enumeration)"],
    "name" : "Local Government Areas (2022 Boundaries) (EN)",
    "level" : "LGA (EN)",
    "add to" : "Filter"
}

In [154]:
await page.get_by_text("Collapse all").click()

subdirs = iterable_var["subdirs"].copy()

for subdir in subdirs:
    await page.get_by_title(subdir).click()
    await page.get_by_title(subdir).click()
    #await page.locator("//*[contains(@class, 'treeNodeElement')]", has = page.get_by_title(subdir)).locator("//*[contains(@class, 'treeNodeExpander collapsed')]").click()
    #await page.locator("//*[contains(@class, 'treeNodeElement')]", has = page.get_by_title(subdir)).locator("//*[contains(@class, 'treeNodeExpander collapsed')]").click()

In [153]:
#await page.locator("//*[contains(@class, 'treeNodeElement')]", has = page.get_by_title(iterable_var["name"], exact=True)).locator("//*[contains(@class, 'treeNodeExpander collapsed')]").click()
#await page.locator("//*[contains(@class, 'treeNodeElement')]", has = page.get_by_title(iterable_var["name"], exact=True)).locator("//*[contains(@class, 'treeNodeExpander collapsed')]").click()

In [155]:
await page.locator("//*[contains(@class, 'treeNodeElement')]", has = page.get_by_title(iterable_var["name"], exact=True)).locator("//*[contains(@class, 'treeNodeExpander collapsed')]").click()

In [157]:
lga0 = [lgas[0]]

In [165]:
states = ["New South Wales", "Victoria", "Queensland", "South Australia", "Western Australia", "Tasmania", "Northern Territory", "Australian Capital Territory"]

In [166]:
for lga in lga0:
    for state in states:
        await page.locator("//*[contains(@class, 'treeNodeElement')]", has = page.get_by_title(state)).locator("//*[contains(@class, 'treeNodeExpander collapsed')]").click()

    await page.locator("//*[contains(@class, 'treeNodeElementBody')]", has = page.get_by_title(lga)).locator("//*[contains(@type, 'checkbox')]").check()

    await page.get_by_title(button_titles[iterable_var["add to"]]).click()

    await page.wait_for_load_state()

    await page.locator("//*[contains(@class, 'downloadButton activeButton')]").click()

    await page.wait_for_load_state()

    await page.locator("//*[contains(@class, 'activeTextField')]").fill("LGA_{}".format(lga))

    await page.locator("//*[contains(@class, 'activeButton')]", has = page.get_by_text("Queue table")).click()
    
    await page.locator("//*[contains(@class, 'treeNodeElementBody')]", has = page.get_by_title(lga)).locator("//*[contains(@type, 'checkbox')]").uncheck()

    for state in states:
        await page.locator("//*[contains(@class, 'treeNodeElement')]", has = page.get_by_title(state)).locator("//*[contains(@class, 'treeNodeExpander expanded')]").click()


TimeoutError: Locator.click: Timeout 30000ms exceeded.
Call log:
  - waiting for get_by_title("Add selected items to the table filter")
  -     - locator resolved to <input type="submit" value="Filter" disabled="disabled" id="buttonForm:addS" name="buttonForm:addS" class="inactiveButton" onclick="return false;" title="Add selected items to the table filter"/>
  -   - attempting click action
  -     2 × waiting for element to be visible, enabled and stable
  -       - element is not enabled
  -     - retrying click action
  -     - waiting 20ms
  -     - waiting for element to be visible, enabled and stable
  -   - element was detached from the DOM, retrying
  -     - locator resolved to <input type="submit" value="Filter" disabled="disabled" id="buttonForm:addS" name="buttonForm:addS" class="inactiveButton" onclick="return false;" title="Add selected items to the table filter"/>
  -   - attempting click action
  -     2 × waiting for element to be visible, enabled and stable
  -       

In [164]:
await page.locator("//*[contains(@class, 'treeNodeElementBody')]", has = page.get_by_title("Albury")).locator("//*[contains(@type, 'checkbox')]").uncheck()

In [ ]:
for var in constant_vars:
    #var = "var1"
    await page.get_by_text("Collapse all").click()
    
    subdirs = constant_vars[var]["subdirs"].copy()

    for subdir in subdirs:
        #await page.get_by_title(subdir).click()
        await page.locator("//*[contains(@class, 'treeNodeElement')]", has = page.get_by_title(subdir)).locator("//*[contains(@class, 'treeNodeExpander collapsed')]").click()

    await page.locator("//*[contains(@class, 'treeNodeElement')]", has = page.get_by_title(constant_vars[var]["name"])).locator("//*[contains(@class, 'levelSelectButton')]").click()
    #await page.locator("//*[contains(@class, 'treeNodeElement')]", has = page.get_by_title(constant_vars[var]["name"])).locator("//*[contains(@class, 'levelSelectButton')]").click()

    await page.locator("//*[contains(@class, 'levelSelectList')]", has = page.get_by_text(constant_vars[var]["level"])).get_by_text(constant_vars[var]["level"]).click()

    await page.get_by_title(button_titles[constant_vars[var]["add to"]]).click()

    await page.wait_for_load_state()

    await page.get_by_text("Untick all").click()

    await page.get_by_text("Collapse all").click()